In [7]:
from scipy.stats import special_ortho_group
import numpy as np
import random
import copy
import math
import sys
import os 
from BasicFunctions import *

np.set_printoptions(precision=4, floatmode='maxprec')
np.set_printoptions(suppress=True)
from graphviz import Graph
from graphviz import Digraph

In [43]:
def WriteNetwork(Ind, nNode, nLayer, count, tidyvisual, correction_v, aling_center):
    #dg = Digraph(format='png')
    dg = Digraph(format='png')
    if tidyvisual:
        dg.attr('node', shape='circle', label='')
    else:
        dg.attr('node', shape='circle')
        
    active_node_arr = Active_node(Ind, "test")
    #Arrange Nodes
    matrices = Ind.getGenom()
    for layer in range(nLayer):
        for node in range(nNode):
            width = active_node_arr[layer][node]
            name = str(layer)+str(node)
            
            if tidyvisual:
                if width:
                    dg.node(name, penwidth=str(width), style='filled', fillcolor='skyblue')
                elif not aling_center:
                    dg.node(name, penwidth=str(width))
            else:
                dg.node(name, penwidth=str(width))
            
    dg.attr(label="Generation:" + str(count) + "\l" + "fitness:" + str(np.round(Ind.getEvaluation(),4)))
    
    #if count%1000 <= 200:
    #    label_ = "Goal matrix change"
    #else:
    #    label_ = "     "
    label_ = "     "
    
    dg.attr(label="Fitness: " + str(np.round(Ind.getEvaluation(),2)) + 
            "   Generation: " + str(count) + "\n" + label_)
    #dg.node("test", label="Goal matrix change", penwidth=0.1)
    dg.attr(fontsize='16')
    #Write links
    for layer in range(1, nLayer):
        for postnode in range(nNode):
            #row
            post = str(layer) + str(postnode)
            for prenode in range(nNode):
                #column
                pre= str(layer-1) + str(prenode)
                intensity =  matrices[layer-1][postnode, prenode]
                if intensity < 0:
                    color_ = "blue"
                    intensity = abs(intensity)
                else:
                    color_ = "black"
                    
                if tidyvisual:
                    if active_node_arr[layer][postnode] & active_node_arr[layer-1][prenode]:
                        if intensity > correction_v:
                            intensity = correction_v
                        dg.edge(pre, post,  penwidth=str(intensity), arrowsize="0.5", color=color_)
                    elif not aling_center:
                        dg.edge(pre, post,  penwidth='0', arrowsize="0", color=color_)
                else:
                    dg.edge(pre, post,  penwidth=str(intensity), arrowsize="0", color=color_)
    return(dg)


def CreateVirtualDuplicateInd(Ind, DesiredGoal):
    zeros = np.full((nNode,nNode),0.001)
    DuplicateNet = list()
    for LayerInt in Ind.getGenom():
        DuplicateNet.append(np.block([[LayerInt,zeros],[zeros,zeros]]))
    DuplicateNet = np.array(DuplicateNet)
    duplicate_genom = genom(DuplicateNet, evaluation(DuplicateNet, DesiredGoal))
    return duplicate_genom

def Duplicate(network, nNode): 
    DuplicateNet = list()
    for LayerInt in network:
        DuplicateNet.append(np.block([[LayerInt,LayerInt],[LayerInt,LayerInt]]))
    DuplicateNet = np.array(DuplicateNet)
    return DuplicateNet


In [55]:
current_generation_individual_group[0].getGenom().shape

(4, 12, 12)

In [69]:
global POPULATION_SIZE
global MAX_GENERATION
global nMatrix
global MUTATION_RATE
global nNode
global nLayer
global nMatrix

POPULATION_SIZE = 100
MAX_GENERATION = 50000#120000
nNode=6
nLayer=5
nMatrix = nLayer-1
MUTATION_RATE = 0.2/(nMatrix*nNode*nNode)


#### Simulation parameters #####

### Active node definition
### 0. Relative contribution of each node to the fitness.
### 1. Relative contribution of each node to the total in-out relation.
### 3. Relative strength of maximal interaction of each node.
ActiveNodeDefinition = 0

### GoalMatrixParameters
GoalMatrixRank = 6
GoalMatirxNorm = 60
GoalMatrixVariance = 10 # If this set to np.nan, the variance is not normalized.

### Goal matrix chnage
GoalMatrixChange = False

### Goal matrix expansion
GoalExpansion = False
DuplicateTime = 1000

norm = 0.01

## Visualization
CreateGif = False
if CreateGif:
    work_dir = "/Users/itoutouma/Aokiken_jupyter/BowTieEvo/temporal_stored_data/"
    # SET the path to your worling directory
    try:
        PRJ_TITLE = "Rank6_goal_fluctuationtest4"
        SavePath = work_dir + PRJ_TITLE
        os.chdir(work_dir)
        os.makedirs(f"{work_dir}{PRJ_TITLE}/result/")
        print(os.getcwd())
    except:
        print("File exist")
    

In [70]:
Define_global_value_in_modules(nNode)
DesiredGoal = CreateRandomGoalMatrix(GoalMatrixRank, norm = GoalMatirxNorm, zvar = GoalMatrixVariance)
print(f"Goal matrix:\n{DesiredGoal}")
print("size{}".format(np.linalg.norm(DesiredGoal, "fro")))
print("rank{}".format(np.linalg.matrix_rank(DesiredGoal)))

Define_global_value_in_modules(nNode, nLayer, GoalMatrixRank, POPULATION_SIZE, MUTATION_RATE, DesiredGoal, ActiveNodeDefinition)
        
current_generation_individual_group_pre = [CreateInd(nNode,nMatrix,norm) for i in range(POPULATION_SIZE)]
current_generation_individual_group = [set_eva(Ind, DesiredGoal) for Ind in current_generation_individual_group_pre]
initial_network_size = np.mean([np.linalg.norm(Total_in_out(Ind.getGenom()), 'fro') for Ind in current_generation_individual_group])
PreGenFitness = 0

TIME_SERIES_NETWORK = list()

print("\nGA: start")
for count in range(MAX_GENERATION):
    
    if GoalMatrixChange & (count%1000 == 0):
        print("Goal Change")
        DesiredGoal =  CreateRandomGoalMatrix(GoalMatrixRank, norm = GoalMatirxNorm, zvar = GoalMatrixVariance)
        current_generation_individual_group = [set_eva(Ind, DesiredGoal) for Ind in current_generation_individual_group]
        Define_global_value_in_modules(GoalMatrixRank_ = GoalMatrixRank, DesiredGoal_ = DesiredGoal)
    
    if GoalExpansion & (count == DuplicateTime):
        #Duplicate
        print("Duplicate")
        zeros = np.full((nNode,nNode),0.001)
        DesiredGoal = Duplicate([DesiredGoal], nNode)[0]
        current_generation_individual_group = [
            CreateVirtualDuplicateInd(current_generation_individual_group[i], DesiredGoal) for i in range(POPULATION_SIZE)]
        
        nNode = nNode*2
        mutation_rate = 0.2/(nMatrix*nNode*nNode)
        Define_global_value_in_modules(nNode_ = nNode, GoalMatrixRank_ = GoalMatrixRank, 
                                       MUTATION_RATE_=mutation_rate, DesiredGoal_ = DesiredGoal)
        
        print("Goal matrix:{}".format(DesiredGoal.shape))
        print("Network:{}\n".format(Total_in_out(current_generation_individual_group[0].getGenom()).shape))

    
    next_generation_network = [copy.deepcopy(Ind.getGenom()) for Ind in current_generation_individual_group]
    next_generation_individual_group = [genom(Network, 0) for Network in next_generation_network]
    next_generation_individual_group_mutated = [add_mutation(Ind, MUTATION_RATE) for Ind in next_generation_individual_group]
    next_generation_individual_group_evaluated = [set_eva(Ind, DesiredGoal) for Ind in  next_generation_individual_group_mutated]
    #Selection
    mixed_population = current_generation_individual_group + next_generation_individual_group_evaluated
    #tournament size is 4
    selected_group = select(mixed_population, 4)
    #Generation change
    selected_net = [copy.deepcopy(Ind.getGenom()) for Ind in selected_group]
    selected_eva = [Ind.getEvaluation() for Ind in selected_group]
    current_generation_individual_group = [genom(selected_net[i], selected_eva[i]) for i in range(POPULATION_SIZE)]
    most_fitted = sorted(selected_group, reverse=True, key=lambda u: u.evaluation)[0]
    
    if count%100==0:
        most_fitted = sorted(selected_group, reverse=True, key=lambda u: u.evaluation)[0]
        ConsActiveNode = Active_node(most_fitted, mode="result")
        print("Gen{}, Loss:{}, Active node:\n{}".format(count, np.mean(selected_eva), ConsActiveNode))
        if CreateGif:
            tidyvisual = True
            align_center = False
            correction_v=15
            G = WriteNetwork(most_fitted, nNode, nLayer, count, tidyvisual, correction_v, align_center)
            Gtitle = SavePath +"/"+  str((count)) + "Gen"
            G.render(Gtitle)
           

Goal matrix:
[[14.2233 10.37    5.0072  4.8752 13.2917  6.2841]
 [ 7.5641 12.456   6.2586  8.9384 13.9919  9.3231]
 [11.9569  6.4452 10.4912  4.8202  9.8188 13.7441]
 [ 6.2967  7.7693 11.8308 15.4121  6.607   8.7543]
 [11.2933 13.7891  7.917   7.4886  7.1335 11.4806]
 [ 5.1348  7.7128 14.7327 12.3715  5.9761  9.9659]]
size60.0
rank6

GA: start
Gen0, Loss:-3598.892143963479, Active node:
[6, 6, 6, 6, 6]
Gen100, Loss:-3598.5447965114054, Active node:
[6, 6, 6, 6, 6]
Gen200, Loss:-3598.1057243629707, Active node:
[6, 6, 6, 6, 6]
Gen300, Loss:-3597.472602717366, Active node:
[6, 6, 6, 6, 6]
Gen400, Loss:-3596.236767091598, Active node:
[6, 6, 6, 6, 6]
Gen500, Loss:-3594.208860835939, Active node:
[6, 6, 6, 6, 6]


KeyboardInterrupt: 

### create gif animation

In [18]:
import shutil
filelist = os.listdir(SavePath)
print(SavePath)
mvfiles = [s for s in filelist if "Gen.png" in s]
mvfilespath = [SavePath + "/"  + s for s in mvfiles]
for file in mvfilespath:
    shutil.move(file, SavePath + '/result')

/Users/itoutouma/Aokiken_jupyter/BowTieEvo/temporal_stored_data/Rank6_goal_fluctuationtest4


In [19]:
 from PIL import Image
import glob
import re
import os
 
def create_gif(in_dir, out_filename):
    print(*[in_dir, '*'])
    path_list = glob.glob(os.path.join(*[in_dir, '*']))
    sort_path_list = sorted(path_list , key=lambda x:int((re.search(r"[0-9]+", x)).group(0)))
    print(sort_path_list)
    imgs = []                                                   # 画像をappendするための空配列を定義
 
    # ファイルのフルパスからファイル名と拡張子を抽出
    for i in range(len(sort_path_list)):
        img = Image.open(sort_path_list[i])                          # 画像ファイルを1つずつ開く
        imgs.append(img)                                        # 画像をappendで配列に格納していく
    imgs[0].save(out_filename, save_all=True, append_images=imgs[1:], optimize=False, duration=200, loop=1)


os.chdir(SavePath)
create_gif(in_dir="./result", out_filename=SavePath + '/MostFitEvo3.gif')

./result *
['./result/0Gen.png', './result/100Gen.png', './result/200Gen.png', './result/300Gen.png']
